During this lab, we will refresh the notions about Homomorphic Encryption seen in class and create a simple Logistic Regression classifier which exploit this kind of encryption for carrying private computations!

This lab is based on the tutorials that can be found on the [TenSEAL github project](https://github.com/OpenMined/TenSEAL/tree/main/tutorials).

# 00 - Homomorphic Encryption

Let's start with recalling the definition of ***Homomorphism***:

A homomorphism is a map between two algebraic structures of the same type (e.g. vector spaces), that preserves the operations of the structures. This means a map $f:\mathcal{A}\to \mathcal{B}$ between two sets $\mathcal{A}$, $\mathcal{B}$ equipped with the same structure such that, if $\star$ is an operation of the structure (supposed here, for simplification, to be a binary operation), then:
$$
f(A_1\star A_2) = f(A_1)\star f(A_2)\;\;\;\forall A_1,A_2\in\mathcal{A}
$$


Now, suppose that $f$ is an encryption map that transform a plaintext which contains sensitive information into a cyphertext capable of completely masking it. This kind of encryption could allow us to work on cyphertexts seamelessly as we would perform the required operations on the original plaintexts!

For example, lets consider the pseudo-code snippet below:
```
# ----- User has these private variables and a secet key
private_x = 453
private_y = 590
secr_key = 42

# ----- User encrypts the variables and send them to an untrusted server for computations
encr_x = HE.encrypt(private_x, secr_key)
encr_y = HE.encrypt(private_y, secr_key)
send_to_remote_server(encr_x, encr_y)

# ----- Server-side: compute the required operations and send the results back to the user
encr_x, encr_y = receive_from_user()
encr_res = encr_x + encr_y
send_to_user(encr_res)

# ----- User receive the results from server and check the correctness of the outsorced operations
encr_res = receive_from_remote_server()
decr_res = encr_sum.decrypt(secr_key)

plain_sum = private_x + private_y

plain_sum == decr_res
>>> prints True
```

Historically, Homomorphic Encryption (from now on, HE) can be divided in three families, depending on the oparations and the number of computations allowed:
* ***Partial HE*** benefits from an unlimited number of computations, but only
one operation is allowed (e.g. summation, multiplication);
* ***Somewhat HE*** allows for multiple operations but suffers from a limited number of computations due to an increasing amount of computations-derived noise;
* ***Full HE*** allows both a multiple number of operations and an unlimited number of computations, but generally suffers from huge computational costs.



In this tutorial we will exploit the python library for HE [TenSEAL](https://github.com/OpenMined/TenSEAL) curated by [OpenMined](https://www.openmined.org/).
TenSEAL is built on top of [Microsoft SEAL](https://github.com/Microsoft/SEAL), a C++ library implementing the BFV (for computations on integer number) and CKKS (on real numbers) homomorphic encryption schemes.

**Let's get started!**

# 01 - Getting Familiar with TenSeal

As always, install the library!

In [1]:
try:
    import tenseal as ts
except ImportError:
    %pip install tenseal
    import tenseal as ts

## 01a - TenSEAL Context

First, we need to instantiate a tenseal context, that is a special object that wrap all the encryption parameters for the selected scheme. In particulat, note how it holds both the secret and public encryption keys.

In [2]:
help(ts.Context)

Help on class Context in module tenseal.enc_context:

class Context(builtins.object)
 |  Context(scheme: tenseal.enc_context.SCHEME_TYPE = None, poly_modulus_degree: int = None, plain_modulus: int = None, coeff_mod_bit_sizes: List[int] = [], encryption_type: tenseal.enc_context.ENCRYPTION_TYPE = <ENCRYPTION_TYPE.ASYMMETRIC: <ENCRYPTION_TYPE.ASYMMETRIC: 0>>, n_threads: int = None, data: _tenseal_cpp.TenSEALContext = None)
 |  
 |  Methods defined here:
 |  
 |  __copy__(self) -> 'Context'
 |  
 |  __init__(self, scheme: tenseal.enc_context.SCHEME_TYPE = None, poly_modulus_degree: int = None, plain_modulus: int = None, coeff_mod_bit_sizes: List[int] = [], encryption_type: tenseal.enc_context.ENCRYPTION_TYPE = <ENCRYPTION_TYPE.ASYMMETRIC: <ENCRYPTION_TYPE.ASYMMETRIC: 0>>, n_threads: int = None, data: _tenseal_cpp.TenSEALContext = None)
 |      Construct a context that holds keys and parameters needed for operating
 |      encrypted tensors using either BFV or CKKS scheme.
 |      
 |     

In [3]:
context = ts.context(ts.SCHEME_TYPE.BFV, poly_modulus_degree=4096, plain_modulus=1032193)
context

## 01b - Encryption

Once we have defined a TenSEAL context and the relative public key, we can encrypt a vector (or a matrix) of numbers (integer or real depending on the exploited encryption scheme).

In [4]:
import numpy as np

In [5]:
plain_vct = [1,2,3,4]
plain_mtx = np.array([1,2,3,4,5,6]).reshape(2,-1)

In [6]:
# ----- encrypted vector
encr_vct = ts.bfv_vector(context, plain_vct)
encr_vct.size()

4

In [7]:
# ----- encrypted matrix
encr_mtx = ts.bfv_tensor(context, plain_mtx)
encr_mtx.shape

[2, 3]

## 02c - Evaluation

Let's start doing some encrypted computations!

In [8]:
# ----- Sum with plain vector
pl_add = encr_vct + plain_vct
pl_add.decrypt()

[2, 4, 6, 8]

In [9]:
# ----- Multiplication with plain vector
pl_add = encr_vct * plain_vct
pl_add.decrypt()

[1, 4, 9, 16]

In [10]:
# ----- Multiplication with encrypted vector
pl_add = encr_vct * encr_vct
pl_add.decrypt()

[1, 4, 9, 16]

In [11]:
# ----- matrix multiplication
encr_mm = encr_mtx.mm(plain_mtx.T)
encr_mm.decrypt().tolist()

[[14, 32], [32, 77]]

## 03c - Requirements

As seen in class, the major drawbacks of performing encrypted computation with HE is a big increase in the computational requirements, both in terms of time and memory! Let's try to measure the difference.

In the example below we will simulate the memory and time impact of performing a backpropagation round of a data matrix with 50 samples and 10 features.

In [12]:
def random_data(n=50, d=10):
    # data separable by the line `y = x`
    x = np.random.randn(n, d)
    y = (x[:, 0] >= x[:, 1]).astype(float)
    return x, y

In [13]:
# ----- Memory requirements
import sys

n = 50
d = 10
plain_mtx, plain_trg = random_data(n, d)
print(f"The memory impact of the plaintext matrix is {sys.getsizeof(plain_mtx)} bytes")

The memory impact of the plaintext matrix is 4120 bytes


In [14]:
poly_mod_degree = 8192
coeff_mod_bit_sizes = [40, 20, 20, 40]
# create TenSEALContext
context = ts.context(ts.SCHEME_TYPE.CKKS,
                     poly_modulus_degree=poly_mod_degree,
                     coeff_mod_bit_sizes=coeff_mod_bit_sizes)
context.global_scale = 2**20
context.generate_galois_keys()

In [15]:
encr_mtx = ts.ckks_tensor(context, plain_mtx)
encr_trg = ts.ckks_tensor(context, plain_trg)

byt = sum([sys.getsizeof(c.data()) for c in encr_mtx.ciphertext()])
print(f"The memory impact of the cyphertext matrix is {byt} bytes")

The memory impact of the cyphertext matrix is 16000 bytes


We discovered that the requirements of simply loading this encrypted data to memory are around 4 times the original plain matrix.
This, of course, depend on the privacy strength that we want to guarantee for our data and algorithm: simpler algorithms or weaker privacy guarantees can work with smaller cyphertexts that will imply a smaller memory overhead.

Let's measure the time impact for performing a simple linear backpropagation round.

In [16]:
import time

class SquareError:
    @staticmethod
    def risk(pred_out, desired_out):
        return ((pred_out-desired_out)**2).sum()

    @staticmethod
    def delta(pred_out, desired_out):
        return (pred_out-desired_out)

class linearClassifier:
    def __init__(self, n_feat, rnd_seed=42):
        self.w = np.random.default_rng(rnd_seed).standard_normal(n_feat)

    def forward(self, x):
        res = x.dot(self.w)
        return res

    def backpropagate(self, x, y, loss_criterion, lr=1e-1):
        # predict
        pred = self.forward(x)
        # compute gradients
        loss_delta = loss_criterion.delta(pred, y)
        delta_w = x.transpose().dot(loss_delta)
        if isinstance(delta_w, ts.CKKSTensor):
            delta_w = np.array(delta_w.decrypt().tolist())
        # update weights
        self.w -= lr * delta_w

In [17]:
lin_cl = linearClassifier(d)
loss = SquareError()
print(f"Pre-backpropagation loss: {loss.risk(lin_cl.forward(plain_mtx), plain_trg):.2f}")
t = time.time()
lin_cl.backpropagate(plain_mtx, plain_trg, loss, lr=1e-2)
t = time.time() - t
print(f"Post-backpropagation loss: {loss.risk(lin_cl.forward(plain_mtx), plain_trg):.2f}")
print(f"\nBackpropagation on plaintext took {t:.2e}")

Pre-backpropagation loss: 341.82
Post-backpropagation loss: 111.09

Backpropagation on plaintext took 1.42e-04


In [18]:
lin_cl = linearClassifier(d)
loss = SquareError()
print(f"Pre-backpropagation loss: {loss.risk(lin_cl.forward(encr_mtx), encr_trg).decrypt().tolist():.2f}")
t = time.time()
lin_cl.backpropagate(encr_mtx, encr_trg, loss, lr=1e-2)
t = time.time() - t
print(f"Post-backpropagation loss: {loss.risk(lin_cl.forward(encr_mtx), encr_trg).decrypt().tolist():.2f}")
print(f"\nBackpropagation on cyphertext took {t:.2e}")

Pre-backpropagation loss: 470.44
Post-backpropagation loss: 126.39

Backpropagation on cyphertext took 2.34e+00


This simple linear example demonstrates that, even for a small number of samples, the overhead in computational time for the backpropagation is around 3-4 orders of magnitude!

If you want to learn more on the theory behind the CKKS encryption scheme, [check this blog post by OpenMined](https://blog.openmined.org/ckks-explained-part-1-simple-encoding-and-decoding/).

# 02 - Implementing a Logistic Regression classifier

Now we will try to implement a simple logistic regression classifier that is able to mask sensitive information thanks to the use of homomorphic encryption.

We will build our classifier so that it will mimic the functionalities of a pytorch module.

## 02a - The Dataset

But first we will download the data for our experiment: [Heart disease](https://www.kaggle.com/datasets/dileep070/heart-disease-prediction-using-logistic-regression?resource=download)

The dataset is publically available on the Kaggle website, and it is from an ongoing cardiovascular study on residents of the town of Framingham, Massachusetts. The classification goal is to predict whether the patient has 10-year risk of future coronary heart disease (CHD).The dataset provides the patients’ information. It includes over 4,000 records and 15 attributes. Since it contains the patients information, it serves well as a sensitive application where we should mask the data and guarantee patients' privacy.

In [19]:
!gdown 1b1cpGagEBnGSscdKK7G0CeyDrlQXUbem

Downloading...
From: https://drive.google.com/uc?id=1b1cpGagEBnGSscdKK7G0CeyDrlQXUbem
To: /content/framingham.csv
100% 196k/196k [00:00<00:00, 91.4MB/s]


In [20]:
import pandas as pd
import numpy as np

In [21]:
def split_train_test(x, y, test_ratio=0.3, num_samples=100):
    rng = np.random.default_rng(42)
    idxs = rng.choice(len(x), size=num_samples, replace=False)
    # delimiter between test and train data
    delim = int(len(idxs) * test_ratio)
    test_idxs, train_idxs = idxs[:delim], idxs[delim:]
    return x[train_idxs], y[train_idxs], x[test_idxs], y[test_idxs]

def heart_disease_data():
    data = pd.read_csv("./framingham.csv")
    # drop rows with missing values
    data = data.dropna()
    # drop some features
    data = data.drop(columns=["education", "currentSmoker", "BPMeds", "diabetes", "diaBP", "BMI"])
    # balance data
    grouped = data.groupby('TenYearCHD')
    data = grouped.apply(lambda x: x.sample(grouped.size().min(), random_state=73).reset_index(drop=True))
    # extract labels
    y = np.array(data["TenYearCHD"].values, dtype=float)[:,None]
    data = data.drop(columns="TenYearCHD")
    # standardize data
    data = (data - data.mean()) / data.std()
    x = np.array(data.values, dtype=float)
    # reduce cardinality
    return split_train_test(x, y, test_ratio=0.3, num_samples=150)

In [22]:
x_train, y_train, x_test, y_test = heart_disease_data()

n_feat = x_train.shape[1]

print(f"x_train has shape: {x_train.shape}")
print(f"y_train has shape: {y_train.shape}")
print(f"x_test has shape: {x_test.shape}")
print(f"y_test has shape: {y_test.shape}")

x_train has shape: (105, 9)
y_train has shape: (105, 1)
x_test has shape: (45, 9)
y_test has shape: (45, 1)


## 02b - Defining and training a standard Logistic Regression classifier

First of all we need to define the logistic (Sigmoid) function for performing the classification task and the loss function for training our model

In [23]:
class Sigmoid:
    @staticmethod
    def forward(x):
        return 1/(1+np.e**(-x))

    @staticmethod
    def prime(x):
        return Sigmoid.forward(x)*(1-Sigmoid.forward(x))

class BinaryCrossEntropyLoss:
    @staticmethod
    def forward(pred, true):
        return (true*np.log(pred) + (1-true)*np.log(1-pred)).mean()

    @staticmethod
    def delta(pred, true):
        return pred - true # cross-entropy + sigmoid trick

def sigmoidal_accuracy_score(pred, true):
    return round((np.abs(pred - true) < .5).astype(float).mean(), 2)

Now, we can define the class for our Logistic Regression classifier:

In [24]:
class LogisticRegression:
    def __init__(self, n_features, rnd_seed=42):
        rng = np.random.default_rng(rnd_seed)
        self.weights = rng.standard_normal((n_features,1))
        self.bias = np.array([0.])

        self._delta_w = 0
        self._delta_b = 0
    
    def forward(self, x):
        raw = x.dot(self.weights) + self.bias
        log = Sigmoid.forward(raw)
        return log

    def predict(self, x):
        return (self.forward(x) > .5).astype(int)

    def reset_gradients(self):
        self._delta_w = 0
        self._delta_b = 0

    def train(self, tr_x, tr_y, vl_x, vl_y, loss, epochs=10, lr=1e-1):
        acc = sigmoidal_accuracy_score(self.predict(vl_x), vl_y)
        print(f"Accuracy @ep 0 = {acc}")

        for ep in range(epochs):
            # reset gradients counters
            self.reset_gradients()

            # predict and update gradients
            pred = self.forward(tr_x)
            loss_delta = loss.delta(pred, tr_y)
            self._delta_w += tr_x.transpose().dot(loss_delta)
            self._delta_b += loss_delta.sum()

            # update parameters
            self.weights -= lr * self._delta_w
            self.bias -= lr * self._delta_b

            acc = sigmoidal_accuracy_score(self.predict(vl_x), vl_y)
            print(f"Accuracy @ep {ep+1} = {acc}")

In [25]:
lrc = LogisticRegression(n_feat)
bce = BinaryCrossEntropyLoss()
lrc.train(x_train, y_train, x_test, y_test, bce, epochs=20, lr=1e-2)

Accuracy @ep 0 = 0.29
Accuracy @ep 1 = 0.4
Accuracy @ep 2 = 0.38
Accuracy @ep 3 = 0.44
Accuracy @ep 4 = 0.49
Accuracy @ep 5 = 0.56
Accuracy @ep 6 = 0.62
Accuracy @ep 7 = 0.67
Accuracy @ep 8 = 0.64
Accuracy @ep 9 = 0.64
Accuracy @ep 10 = 0.64
Accuracy @ep 11 = 0.64
Accuracy @ep 12 = 0.67
Accuracy @ep 13 = 0.69
Accuracy @ep 14 = 0.69
Accuracy @ep 15 = 0.69
Accuracy @ep 16 = 0.69
Accuracy @ep 17 = 0.69
Accuracy @ep 18 = 0.69
Accuracy @ep 19 = 0.69
Accuracy @ep 20 = 0.69


With this particular configuration, the linear classifier is able to classify correctly almost 70% of the test samples, not bad!

## 02c - Evaluating a trained logistic regression classifier on encrypted data

Before trying to train a classifier on private data from scracth we need to fix some issues.

For example, remember how our encryption scheme for real number (CKKS) allows us to perform only addition and multiplcation? That means we can't use the very core structure of a Logistic regression classifier: the logistic function!

Luckily, the Sigmoid can be approximated by a polynomial of degree 3 that behaves sufficiently well in the range \[-5,5\] (remember that we have standardized our data!):
$$
sigmoid(x) = 0.5 + 0.197 * x - 0.004 * x^3
$$
\[from [Chen2018Logistic](https://eprint.iacr.org/2018/462.pdf)\].

In [26]:
class ApproximatedSigmoid:
    @staticmethod
    def forward(x):
        if isinstance(x, np.ndarray):
            return np.polyval([0.5, 0.197, 0, -0.004], x)
        return x.polyval([0.5, 0.197, 0, -0.004])
    
    @staticmethod
    def prime(x):
        if isinstance(x, np.ndarray):
            return np.polyval([0.197, 0, -0.012], x)
        return x.polyval([0.197, 0, -0.012])

Before defining a Logistic regression classifier that is able to be trained on encrypted data, we can define one that is able to load the parameters from a trained model (on plaintexts) and just make predictions (on encrypted data).
This new model obviously will observe the constraints imposed by the encryption scheme and so it will exploit the polynomial approximation of the Sigmoid.

In [27]:
class EncryptedLogisticRegression:
    def __init__(self, trained_classifier):
        self.weights = trained_classifier.weights
        self.bias = trained_classifier.bias

    def forward(self, x):
        raw = x.dot(self.weights) + self.bias
        log = ApproximatedSigmoid.forward(raw)
        return log

In [28]:
elrc = EncryptedLogisticRegression(lrc)

In [29]:
poly_mod_degree = 8192
coeff_mod_bit_sizes = [40, 21, 21, 21, 21, 21, 21, 40]
context = ts.context(ts.SCHEME_TYPE.CKKS, poly_mod_degree, -1, coeff_mod_bit_sizes)
context.global_scale = 2 ** 21
context.generate_galois_keys()

In [30]:
# ----- Encrypt private data
encr_x_test = ts.ckks_tensor(context, x_test)

In [31]:
# ----- Forward encrypted data through the network
encr_pred_ts = elrc.forward(encr_x_test)

In [32]:
# ----- Decrypt and compute predictions
decr_pred_ts = np.array(encr_pred_ts.decrypt().tolist())
encr_pred_ts = (decr_pred_ts > .5).astype(int)

encr_ts_acc = sigmoidal_accuracy_score(encr_pred_ts, y_test)
print(f"Encrypted test accuracy: {encr_ts_acc}")

Encrypted test accuracy: 0.69


We observe that in this case the random noise added by the encryption scheme was not able to corrupt (or improve) the model average accuracy!

## 02d - Training a Logistic Regression classifier on private data

Now, we can extend the class defined above so that the classifier can be trained on encrypted data from scratch.

In [33]:
class EncryptedLogisticRegression:
    def __init__(self, n_features, rnd_seed=42):
        rng = np.random.default_rng(rnd_seed)
        self.weights = rng.standard_normal(n_features)
        self.bias = np.array([0.])

        self._delta_w = 0
        self._delta_b = 0
    
    def forward(self, x):
        raw = x.dot(self.weights) + self.bias
        log = ApproximatedSigmoid.forward(raw)
        return log

    def predict(self, x):
        return (self.forward(x) > .5).astype(int)

    def reset_gradients(self):
        self._delta_w = 0
        self._delta_b = 0

    def encrypt(self, context):
        self.weights = ts.ckks_vector(context, self.weights)
        self.bias = ts.ckks_vector(context, self.bias)

    def decrypt(self):
        self.weights = np.array(self.weights.decrypt())
        self.bias = np.array(self.bias.decrypt())

    def train(self, tr_x, tr_y, vl_x, vl_y, tr_context, loss, epochs=20, lr=1e-1):
        acc = sigmoidal_accuracy_score(self.predict(vl_x)[:,None], vl_y)
        print(f"Accuracy @ep 0 = {acc}")

        for ep in range(epochs):
            # reset gradients counters
            self.reset_gradients()

            # encrypt model parameters
            self.encrypt(tr_context)

            # predict and update gradients
            for enc_x, enc_y in zip(tr_x, tr_y):
                pred = self.forward(enc_x)
                loss_delta = loss.delta(pred, enc_y)
                self._delta_w += enc_x * loss_delta
                self._delta_b += loss_delta

            # update parameters
            self.weights -= lr * self._delta_w
            self.bias -= lr * self._delta_b

            # decrypt model parameters
            self.decrypt()

            acc = sigmoidal_accuracy_score(self.predict(vl_x)[:,None], vl_y)
            print(f"Accuracy @ep {ep+1} = {acc}")

In [34]:
encr_x_train = [ts.ckks_vector(context, x) for x in x_train]
encr_y_train = [ts.ckks_vector(context, y) for y in y_train]

In [35]:
elrc = EncryptedLogisticRegression(n_feat)
elrc.train(encr_x_train, encr_y_train,
           x_test, y_test,
           context, bce,
           epochs=20, lr=1e-2)

Accuracy @ep 0 = 0.29
Accuracy @ep 1 = 0.47
Accuracy @ep 2 = 0.62
Accuracy @ep 3 = 0.62
Accuracy @ep 4 = 0.6
Accuracy @ep 5 = 0.64
Accuracy @ep 6 = 0.67
Accuracy @ep 7 = 0.64
Accuracy @ep 8 = 0.64
Accuracy @ep 9 = 0.64
Accuracy @ep 10 = 0.67
Accuracy @ep 11 = 0.67
Accuracy @ep 12 = 0.67
Accuracy @ep 13 = 0.67
Accuracy @ep 14 = 0.69
Accuracy @ep 15 = 0.69
Accuracy @ep 16 = 0.69
Accuracy @ep 17 = 0.69
Accuracy @ep 18 = 0.69
Accuracy @ep 19 = 0.69
Accuracy @ep 20 = 0.69


From the results above, we can see how the logistic regressor classifier can be trained on the encrypted data and achieve results comparable to the "plaintext" classifier despite the added encryption noise and computational overhead!